In [1]:
import os
os.chdir("../")
%pwd

'/home/megh/AI/Projects/Grapevine_Leaf_Classification'

In [25]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class TrainingConfig:
    root_dir: Path
    trained_model_path: Path
    updated_base_model_path: Path
    training_data : Path
    params_epoch: int
    params_batch_size: int
    params_is_augmentation: bool
    params_image_size: list

@dataclass(frozen=True)
class PrepareCallbackConfig:
    root_dir: Path
    tensorboard_root_log_dir:Path
    checkpoint_model_filepath: Path



In [26]:
from CNNClassifier.constants import *
from CNNClassifier.utils.common import read_yaml, create_directories
import tensorflow as tf


In [27]:
class ConfigurationManager:
    def __init__(self, 
                 config_filepath = CONFIG_FILE_PATH,
                 params_filepath= PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    def get_prepare_callback_config(self)-> PrepareCallbackConfig:
        config = self.config.prepare_callbacks
        model_ckpt_dir = os.path.dirname(config.checkpoint_model_filepath)

        create_directories([ 
                            Path(model_ckpt_dir),
                            Path(config.tensorboard_root_log_dir)  ])

        prepare_callback_config = PrepareCallbackConfig(
            root_dir = Path(config.root_dir),
            tensorboard_root_log_dir = Path(config.tensorboard_root_log_dir),
            checkpoint_model_filepath= Path(config.checkpoint_model_filepath)
    
        )

        return prepare_callback_config
         
    def get_training_config(self)-> TrainingConfig:
        training = self.config.training
        prepare_base_model = self.config.prepare_base_model 
        params = self.params
        training_data = os.path.join(self.config.data_ingestion.unzip_dir,"Grapevine_Leaves_Image_Dataset")
        create_directories([Path(training.root_dir)])
        
        training__config = TrainingConfig(
                                root_dir= Path(training.root_dir),
                                trained_model_path = Path(training.training_model_path),
                                updated_base_model_path= Path(prepare_base_model.updated_base_model_path),
                                training_data = Path(training_data),
                                params_epoch= params.EPOCHS,
                                params_batch_size = params.BATCH_SIZE,
                                params_is_augmentation= params.AUGMENTATION,
                                params_image_size = params.IMAGE_SIZE)
        
        return training__config
    

    



In [28]:
config = read_yaml(CONFIG_FILE_PATH)
print(config.data_ingestion.unzip_dir)


[2023-08-31 21:08:22,679: INFO: common: yaml fileconfig/config.yamlloaded Successfully]
artifacts/data_ingestion


In [29]:
import time


In [30]:
class PrepareCallback:
    def __init__(self, config : PrepareCallbackConfig):
        self.config = config

    @property
    def _create_tb_callbacks(self):
        timestamp = time.strftime("%Y-%m-%d-%H-%M-%S")
        tb_running_log_dir = os.path.join(self.config.tensorboard_root_log_dir, f"tb_logs_at_{timestamp}",)
        print(tb_running_log_dir)
        return tf.keras.callbacks.TensorBoard(log_dir=tb_running_log_dir)
    
    
    @property
    def _create_ckpt_callbacks(self):
        
        print(self.config.checkpoint_model_filepath)

        return tf.keras.callbacks.ModelCheckpoint(
            filepath= f"{self.config.checkpoint_model_filepath}",
            save_best_only=True)
    
    def get_tb_ckpt_callbacks(self):
        return [self._create_tb_callbacks, self._create_ckpt_callbacks]


In [31]:
import os
import urllib.request as request
from zipfile import ZipFile
import tensorflow as tf                   
import time

In [50]:
class Training:
    def __init__(self, config: TrainingConfig):
        self.config = config

    def get_base_model(self):
        self.model = tf.keras.models.load_model(
            self.config.updated_base_model_path
        )

    def train_valid_generator(self):

        datagenerator_kwargs = dict(
            rescale = 1./225,
            validation_split= 0.20
        )

        dataflow_kwargs = dict(
            target_size = self.config.params_image_size[:-1],
            batch_size = self.config.params_batch_size,
            interpolation = 'bilinear'
        )


        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs)
        
        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory = self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )

        if self.config.params_is_augmentation:
            train_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
                rotation_range = 40,
                horizontal_flip = True,
                width_shift_range=0.2,
                height_shift_range= 0.2,
                shear_range=0.2,
                zoom_range=0.2,
                **datagenerator_kwargs
            )
        else: 
            train_datagenerator = valid_datagenerator

        self.train_generator = train_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset= 'training',
            shuffle=True,
            **dataflow_kwargs
        )

    @staticmethod
    def save_model(path: Path, model: tf.keras.Model):
        model.save(path)

    def train(self, callback_list: list):
        self.steps_per_epoch = self.train_generator.samples// self.train_generator.batch_size 
        self.validation_steps = self.valid_generator.samples// self.valid_generator.batch_size
        print(self.steps_per_epoch)
        print(self.validation_steps)
        self.model.fit(
            self.train_generator,
            epochs=self.config.params_epoch,
            steps_per_epoch = self.steps_per_epoch,
            validation_steps = self.validation_steps,
            validation_data = self.valid_generator,
            callbacks= callback_list
        )

        self.save_model(
            path=self.config.trained_model_path,
            model=self.model
        )

class Training:
    def __init__(self, config: TrainingConfig):
        self.config = config
    
    def get_base_model(self):
        self.model = tf.keras.models.load_model(
            self.config.updated_base_model_path
        )
    
    def train_valid_generator(self):

        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split=0.20
        )

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )

        if self.config.params_is_augmentation:
            train_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
                rotation_range=40,
                horizontal_flip=True,
                width_shift_range=0.2,
                height_shift_range=0.2,
                shear_range=0.2,
                zoom_range=0.2,
                **datagenerator_kwargs
            )
        else:
            train_datagenerator = valid_datagenerator

        self.train_generator = train_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="training",
            shuffle=True,
            **dataflow_kwargs
        )

    @staticmethod
    def save_model(path: Path, model: tf.keras.Model):
        model.save(path)


    def train(self, callback_list: list):
        self.steps_per_epoch = self.train_generator.samples // self.train_generator.batch_size
        self.validation_steps = self.valid_generator.samples // self.valid_generator.batch_size

        self.model.fit(
            self.train_generator,
            epochs=self.config.params_epoch,
            steps_per_epoch=self.steps_per_epoch,
            validation_steps=self.validation_steps,
            validation_data=self.valid_generator,
            callbacks=callback_list

        )


        self.save_model(
            path=self.config.trained_model_path,
            model=self.model
        )

In [51]:
try: 
    config = ConfigurationManager()
    prepare_callbacks_config= config.get_prepare_callback_config()
    prepare_callbacks= PrepareCallback(config = prepare_callbacks_config)
    callback_list = prepare_callbacks.get_tb_ckpt_callbacks()

    training_config = config.get_training_config()
    training= Training(config = training_config)
    training.get_base_model()
    training.train_valid_generator()
    training.train(
      callback_list = callback_list
    )

except Exception as e:
    raise e

[2023-08-31 21:42:14,337: INFO: common: yaml fileconfig/config.yamlloaded Successfully]
[2023-08-31 21:42:14,339: INFO: common: yaml fileparams.yamlloaded Successfully]
[2023-08-31 21:42:14,340: INFO: common: created directory at:artifacts]
[2023-08-31 21:42:14,341: INFO: common: created directory at:artifacts/prepare_callbacks/checkpoint_dir]
[2023-08-31 21:42:14,341: INFO: common: created directory at:artifacts/prepare_callbacks/tensorboard_log_dir]
artifacts/prepare_callbacks/tensorboard_log_dir/tb_logs_at_2023-08-31-21-42-14
artifacts/prepare_callbacks/checkpoint_dir/model.h5
[2023-08-31 21:42:14,343: INFO: common: created directory at:artifacts/training]
Found 100 images belonging to 5 classes.
Found 400 images belonging to 5 classes.
25
6
25/25 [==============================] - 6s 239ms/step - loss: 24.3647 - accuracy: 0.2050 - val_loss: 18.6317 - val_accuracy: 0.2812


/home/megh/AI/Projects/Grapevine_Leaf_Classification/cnncls/lib/python3.8/site-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [49]:
image = [224,224,3]
print(image[:-1])

[224, 224]
